# Imports

In [4]:
# general purpose libraries
import threading
import os
import random
import typing
import json
import time

# data utils libraries
import math
import numpy as np
from heapq import heappush, heappop
from copy import copy, deepcopy

# vizualization libraries
from tabulate import tabulate

# Construct Classes Architecture

## Helpers

In [5]:
class ComputationHelper:
    
    @staticmethod
    def euclidean_distance(current_position, goal_position):
        oy_squared_distance = math.pow(current_position[0] - goal_position[0], 2)
        ox_squared_distance = math.pow(current_position[1] - goal_position[1], 2)
        return math.sqrt(oy_squared_distance + ox_squared_distance)
    
    @staticmethod
    def manhattan_distance(current_position, goal_position):
        oy_distance = abs(current_position[0] - goal_position[0])
        ox_distance = abs(current_position[1] - goal_position[1])
        return oy_distance + ox_distance

## Map

In [6]:
class Map:
    NORTH, EAST, SOUTH, WEST = (-1, 0), (0, 1), (1, 0), (0, -1)
    DIRECTIONS = [NORTH, EAST, SOUTH, WEST]
    UNKNOWN, WALL, EMPTY, GOAL, AGENT = "?", "X", " ", "G", "A"

    def __init__(self, map_json: dict, unknown_map: bool = True):
        goal_position = tuple(map_json["goal_position"])
        self.height = map_json["height"]
        self.width = map_json["width"]     
        self.name = map_json["name"]     
        self.goal_position = goal_position

        obstacles_positions = []
        for obstacle_position in map_json["obstacles_positions"]:
            obstacles_positions.append(tuple(obstacle_position))

        if unknown_map:
            self.values = self.__compute_unknown_map(goal_position)
        else:
            self.values = self.__compute_final_map(goal_position, obstacles_positions)

    def __compute_final_map(self, goal_position: typing.Tuple[int, int], obstacles_positions: typing.List[typing.Tuple[int, int]]) -> list:
        result = np.array([[Map.EMPTY for _ in range(self.width)] for _ in range(self.height)], dtype=str)
        for obstacle_position in obstacles_positions:
            result[obstacle_position] = Map.WALL

        result[goal_position] = Map.GOAL

        return result

    def __compute_unknown_map(self, goal_position: typing.Tuple[int, int]) -> typing.List[str]:
        result = np.array([[Map.UNKNOWN for _ in range(self.width)] for _ in range(self.height)], dtype=str)
        result[goal_position] = Map.GOAL
        return result
    
    def get_random_empty_position(self) -> typing.Tuple[int, int]:
        empty_positions = []
        for row_index in range(0, len(self.values)):
            current_row = self.values[row_index]
            for column_index in range(0, len(current_row)):
                current_cell_value = self.values[row_index][column_index]
                if current_cell_value == Map.EMPTY:
                    empty_positions.append(tuple([row_index, column_index]))
        
        return random.choice(empty_positions)

    def update_map(self, position: typing.Tuple[int, int], new_value: str) -> None:
        current_value = self.values[position]
        if current_value in [Map.WALL, Map.GOAL]:
            raise Exception("Cannot update map!")
        
        self.values[position] = new_value
    
    def get_goal_position(self):
        return self.goal_position

    def get_next_valid_positions(self, current_position: typing.Tuple[int, int]) -> typing.List[typing.Tuple[int, int]]:
        valid_positions = []
        for direction in Map.DIRECTIONS:
            new_possible_position = self.__compute_new_position(current_position, direction)
            if (self.__is_position_valid(new_possible_position)):
                valid_positions.append(new_possible_position)
        
        return valid_positions
    
    def __compute_new_position(self, position, direction):
        return (position[0] + direction[0], position[1] + direction[1])

    def __is_position_valid(self, position):
        line_condition = position[0] >= 0 and position[0] < self.height
        column_condition = position[1] >= 0 and position[1] < self.width

        if line_condition and column_condition:
            return self.values[position] in [Map.UNKNOWN, Map.EMPTY, Map.GOAL, Map.AGENT]

        return False

    def print_map(self) -> None:
        print("Map: {}".format(self.name))
        print(tabulate(self.values, headers='keys', tablefmt='fancy_grid', showindex=True))



In [7]:
class MapHelper:
    
    @staticmethod
    def combine_maps(map_to_be_updated: Map, map_with_new_information: Map) -> Map:
        result_map = deepcopy(map_to_be_updated)

        for row_index in range(len(result_map.values)):
            for column_index in range(len(result_map.values[row_index])):
                actual_value = result_map.values[row_index][column_index]
                if actual_value == Map.UNKNOWN:
                    possible_new_value = map_with_new_information.values[row_index][column_index]
                    if possible_new_value == Map.AGENT:
                        result_map.values[row_index][column_index] = Map.EMPTY
                    else:
                        result_map.values[row_index][column_index] = possible_new_value

        return result_map
        
    @staticmethod
    def compute_information_gain_percentage(agent_map : Map, new_possible_common_map: Map) -> float:
        agent_information_gain = 0

        for row_index in range(len(agent_map.values)):
            for column_index in range(len(agent_map.values[row_index])):
                agent_percepted_value = agent_map.values[row_index][column_index]
                new_possible_common_map_value = new_possible_common_map.values[row_index][column_index]
                
                if agent_percepted_value == Map.UNKNOWN \
                        and new_possible_common_map_value in [Map.EMPTY, Map.WALL]:
                    agent_information_gain += 1

        total_information = float(agent_map.width * agent_map.height)
        result = float(agent_information_gain) / total_information
        return result * 100


## Agent

In [8]:
class AgentCommunicationReason:
    CHECK_NEXT_POSITION = 0
    UPDATE_COMMON_MAP_UPON_AGENT_SEARCH_COMPLETION = 1
    UPDATE_COMMON_MAP_UPON_AGENT_REQUEST = 2
    GET_COMMON_MAP = 3

In [9]:
class Agent(threading.Thread):
    def __init__(self, agent_index: int, map: Map, initial_position: typing.Tuple[int, int], world_communicator: typing.Callable) -> None:
        threading.Thread.__init__(self)
        self.agent_name = "Agent #{}".format(agent_index)
        self.world_communicator = world_communicator
        self.position_history = []
        self.current_position = initial_position
        self.map = map
        self.map.update_map(initial_position, Map.AGENT)

    def run(self):
        while True:
            # Update agent's map with common information
            self.update_agent_map_with_common_map()

            # Update with surrounding information
            self.update_surroundings()
            
            # Try to communicate current discoveries
            self.communicate()

            # Predict best possible path to solution
            predicted_path = self.a_star()

            # Stop condition
            if (len(predicted_path) == 1 and predicted_path[0] == self.current_position):
                break
            
            # Follow path if possible
            for possible_next_position in predicted_path:
                if possible_next_position != self.current_position:
                    value_at_position = self.get_real_value_at_new_position(possible_next_position)
                    
                    if value_at_position == Map.WALL:
                        self.map.update_map(possible_next_position, Map.WALL)
                        break
                    else:
                        self.move_to_new_position(possible_next_position)

        self.finalize_execution()


    def communicate(self):
        self.world_communicator(AgentCommunicationReason.UPDATE_COMMON_MAP_UPON_AGENT_REQUEST, self, {})

    def finalize_execution(self):
        print("\n{} finished!".format(self.agent_name))
        self.describe()
        self.update_common_map()

    def update_common_map(self):
        self.world_communicator(AgentCommunicationReason.UPDATE_COMMON_MAP_UPON_AGENT_SEARCH_COMPLETION, self, {})

    def update_agent_map_with_common_map(self):
        common_map = self.get_common_map()
        current_map = self.map

        self.map = MapHelper.combine_maps(current_map, common_map)

    def get_common_map(self) -> Map:
        return self.world_communicator(AgentCommunicationReason.GET_COMMON_MAP, self, {})

    def update_surroundings(self):
        for possible_next_position in self.map.get_next_valid_positions(self.current_position):
            if self.map.values[possible_next_position] == Map.UNKNOWN:
                value_at_position = self.get_real_value_at_new_position(possible_next_position) 
                self.map.update_map(possible_next_position, value_at_position)

    def move_to_new_position(self, new_position: typing.Tuple[int, int]) -> None:
        self.map.update_map(self.current_position, Map.EMPTY)
        self.map.update_map(new_position, Map.AGENT)
        self.position_history.append(self.current_position)
        self.current_position = new_position

    def get_real_value_at_new_position(self, new_position: typing.Tuple[int, int]):
        return self.world_communicator(\
                        AgentCommunicationReason.CHECK_NEXT_POSITION, self, {"position_to_check" : new_position})

    def a_star(self):
        initial_position = self.current_position
        goal_position = self.map.get_goal_position()

        queue = []
        heappush(queue, (-math.inf, initial_position, []))

        while queue:
            queue_item = heappop(queue)
            current_position = queue_item[1]
            current_path = queue_item[2]

            if current_position == goal_position:
                return current_path

            for next_position in self.map.get_next_valid_positions(current_position):
                if next_position not in current_path:
                    updated_path = copy(current_path)
                    updated_path.append(current_position)
                    new_element = (\
                        self.__heuristic(updated_path, next_position, goal_position),\
                        next_position,\
                        updated_path)
                    heappush(queue, new_element)
                
        raise Exception("Investigate map [{}] !".format(self.map.name))

    """ The lower the value returned by the heuristic, the better."""
    def __heuristic(self, updated_path, next_position, goal_position, manhattan = False):
        standard_value = None
        if manhattan:
            standard_value = ComputationHelper.manhattan_distance(next_position, goal_position) \
                + len(updated_path)
        else:
            standard_value = ComputationHelper.euclidean_distance(next_position, goal_position) \
                + len(updated_path)

        credit_score = self.__compute_credit_score(updated_path)
        return standard_value - credit_score
    
    """ We give credit to paths that follow discovered cells."""
    def __compute_credit_score(self, updated_path: typing.List[typing.Tuple[int, int]], credit_parameter: float = 10):
        a_star_predicted_path_credit = 0
        
        for position in updated_path:
            if self.map.values[position] == Map.WALL:
                return math.inf
            if self.map.values[position] == Map.EMPTY and position not in self.position_history:
                a_star_predicted_path_credit += credit_parameter
        
        return a_star_predicted_path_credit

    def describe(self) -> None:
        print("\nAgent name: {}".format(self.agent_name))
        print("current_position: {}".format(self.current_position))
        print("position_history: {}".format(self.position_history))
        self.map.print_map()



## World

In [10]:
class World:
    def __init__(self, map_json: dict, number_of_agents: int, \
            communication_delay: int, information_gain_percentage_threshold:int):
        self.real_map = Map(map_json, unknown_map=False)
        self.common_map = Map(map_json, unknown_map=True)
        self.agents = self.__init_agents(map_json, number_of_agents)
        self.communication_delay = communication_delay
        self.information_gain_percentage_threshold = information_gain_percentage_threshold
        self.thread_lock = threading.Lock()
        self.metrics = {
            "name": map_json["name"],
            "total_communications" : 0,
            "common_map_updates" : 0,
            "communication_delay" : communication_delay,
            "information_gain_percentage_threshold" : information_gain_percentage_threshold,
            "number_of_agents" : number_of_agents,
            "start_time" : None,
            "finish_time" : None,
            }

    def __init_agents(self, map_json: dict, number_of_agents: int) -> typing.List[Agent]:
        agents = []
        for agent_index in range(number_of_agents):
            random_empty_position = self.real_map.get_random_empty_position()
            agents.append(Agent(agent_index, Map(map_json, unknown_map=True), random_empty_position, self.world_communicator))
        
        return agents

    def world_communicator(self, communication_reason: int, agent: Agent, kwargs) -> object:
        if (communication_reason == AgentCommunicationReason.CHECK_NEXT_POSITION):
            return self.check_next_position(agent, **kwargs)
        
        return self.world_communicator_under_bandwith_constraints(communication_reason, agent)
        

    def world_communicator_under_bandwith_constraints(self, communication_reason: int, agent: Agent) -> object:
        # Delay introduced to simulate real life communication
        time.sleep(self.communication_delay) 
        
        # With this lock we coordinate agents communication requests to the World 
        self.thread_lock.acquire()
        self.increment_metric("total_communications")
        response = None

        if communication_reason == AgentCommunicationReason.GET_COMMON_MAP:
            response = deepcopy(self.common_map)
            
        if communication_reason in \
                [AgentCommunicationReason.UPDATE_COMMON_MAP_UPON_AGENT_SEARCH_COMPLETION, \
                    AgentCommunicationReason.UPDATE_COMMON_MAP_UPON_AGENT_REQUEST]:
            response =  self.possibly_update_common_map(agent)
        
        self.thread_lock.release()
        return response

    def possibly_update_common_map(self, agent: Agent) -> None:
        if (self.should_update_common_information(agent)):
            self.update_common_map(agent)

    def should_update_common_information(self, requesting_agent: Agent) -> bool:
        proposed_new_common_map = MapHelper.combine_maps(self.common_map, requesting_agent.map)
        
        global_information_gain = 0
        for agent in self.agents:
            agent_information_gain = MapHelper.compute_information_gain_percentage(agent.map, proposed_new_common_map)
            global_information_gain += agent_information_gain   
        
        global_information_gain /= len(self.agents)
        return global_information_gain >= self.information_gain_percentage_threshold

    def check_next_position(self, agent: Agent, position_to_check: typing.Tuple[int, int]) -> object:
        if int(ComputationHelper.euclidean_distance(agent.current_position, position_to_check)) != 1:
            raise Exception("Agent {} is allowed to check only nearest positions {} {}!".format(agent.agent_name, agent.current_position, position_to_check))

        return self.real_map.values[position_to_check]

    def update_common_map(self, agent: Agent) -> None:       
        new_common_map = MapHelper.combine_maps(self.common_map, agent.map)
        self.common_map = new_common_map
        self.increment_metric("common_map_updates")

        print("\nCommon Map updated by {} !".format(agent.agent_name))
        self.common_map.print_map()

    def increment_metric(self, metric_key: str) -> None:
        self.metrics[metric_key] += 1

    def start(self):
        self.describe()
        
        print("-" * 80)
        print("Releasing agents")
        print("-" * 80)
        
        self.metrics["start_time"] = time.time()
        # Release agents
        for agent in self.agents:
            agent.start()
        
        # Wait for agents
        for agent in self.agents:
            agent.join()
        
        self.metrics["finish_time"] = time.time()
        self.metrics["execution_time"] = self.metrics["finish_time"] - self.metrics["start_time"] 
        
        print("\nAgents finished... printing world state")
        self.describe()
        
        return self.metrics

    def describe(self):
        print("Real map...")
        self.real_map.print_map()

        print("Common map...")
        self.common_map.print_map()

        print("Agents...")
        for agent in self.agents:
            agent.describe()
        

# Load Worlds

In [11]:
maps_folder_path = "./maps"

NUMBER_OF_AGENTS = 5
COMMUNICATION_DELAY = 0.1
INFORMATION_GAIN_PERCENTAGE_THRESHOLD = 5

worlds = []
for map_file_name in os.listdir(maps_folder_path):
    map_file_absolute_path = os.path.join(maps_folder_path, map_file_name)
    file = open(map_file_absolute_path, "r")
    map_json = json.load(file)
    worlds.append(World(map_json, NUMBER_OF_AGENTS, COMMUNICATION_DELAY, INFORMATION_GAIN_PERCENTAGE_THRESHOLD))

# Start searching

In [12]:
# %%capture cap --no-stderr

execution_results = []
for world in worlds:
    execution_result = world.start()
    execution_results.append(execution_result)

Real map...
Map: wall_map
╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ 0   │ 1   │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│  0 │     │     │     │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  1 │     │     │     │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  2 │     │     │     │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  3 │     │     │     │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  4 │     │     │     │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  5 │     │     │     │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼


Common Map updated by Agent #1 !
Map: wall_map
╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ 0   │ 1   │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│  0 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  1 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  2 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │     │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  3 │ ?   │ ?   │ ?   │ ?   │ ?   │     │     │     │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  4 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  5 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │     │
├────┼─────┼──


Agent #1 finished!

Agent name: Agent #1
current_position: (9, 8)
position_history: [(3, 6), (4, 6), (4, 7), (4, 8), (4, 9), (5, 9), (6, 9), (7, 9), (7, 8), (7, 7), (6, 7), (6, 8), (5, 8), (5, 7), (5, 6), (6, 6), (7, 6), (7, 5), (6, 5), (5, 5), (4, 5), (4, 4), (5, 4), (5, 3), (4, 3), (4, 2), (4, 1), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (2, 7), (2, 6), (2, 5), (2, 4), (2, 3), (2, 2), (2, 1), (2, 0), (3, 0), (4, 0), (5, 0), (5, 1), (5, 2), (6, 2), (7, 2), (7, 3), (8, 3), (8, 4), (9, 4), (9, 5), (9, 6), (9, 7)]
Map: wall_map
╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ 0   │ 1   │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│  0 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  1 │     │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼──


Common Map updated by Agent #2 !
Map: x_square_map
╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ 0   │ 1   │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│  0 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  1 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  2 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  3 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │     │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  4 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  5 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │     │     │
├────┼────


Common Map updated by Agent #4 !
Map: x_square_map
╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ 0   │ 1   │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│  0 │ ?   │ ?   │     │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  1 │ ?   │ ?   │ ?   │     │     │ ?   │ ?   │ ?   │ ?   │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  2 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  3 │ ?   │ ?   │ ?   │ ?   │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  4 │ ?   │ ?   │ ?   │ X   │     │ X   │ ?   │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  5 │     │ ?   │     │ ?   │ X   │ ?   │ ?   │ ?   │     │     │
├────┼────

...


Common Map updated by Agent #0 !
Map: complex_map
╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ 0   │ 1   │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│  0 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  1 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  2 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  3 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  4 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  5 │     │ ?   │ ?   │ ?   │ X   │ ?   │     │ ?   │     │     │
├────┼─────


Common Map updated by Agent #4 !
Map: complex_map
╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ 0   │ 1   │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│  0 │     │     │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  1 │     │     │ ?   │     │ ?   │     │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  2 │     │     │     │     │ X   │     │     │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  3 │ ?   │ ?   │ ?   │     │ X   │     │     │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  4 │ ?   │ ?   │     │     │ X   │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  5 │     │ ?   │ ?   │ X   │ X   │ X   │     │ ?   │     │     │
├────┼─────


Common Map updated by Agent #4 !
Map: complex_map
╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ 0   │ 1   │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│  0 │     │     │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  1 │     │     │ ?   │     │     │     │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  2 │     │     │     │     │ X   │     │     │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  3 │ ?   │     │     │     │ X   │     │     │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  4 │     │     │     │     │ X   │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  5 │     │ X   │ X   │ X   │ X   │ X   │     │ ?   │     │     │
├────┼─────

Map: no_obstacles_map
╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ 0   │ 1   │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│  0 │     │     │     │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  1 │     │     │     │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  2 │     │     │     │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  3 │     │     │     │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  4 │     │     │     │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  5 │     │     │     │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼────


Agent #1 finished!

Agent name: Agent #1
current_position: (9, 8)
position_history: []
Map: no_obstacles_map
╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ 0   │ 1   │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│  0 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  1 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  2 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  3 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  4 │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │ ?   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│  5 │ ?   │ ?   │ 

In [13]:
with open("Search.log", "w") as f:
    f.write(cap.stdout)

NameError: name 'cap' is not defined

# Execution Results

In [11]:
for execution_result in execution_results:
    print("-" * 80, "\n")
    print(json.dumps(execution_result, indent=4))
    print("-" * 80, "\n")

-------------------------------------------------------------------------------- 

{
    "name": "wall_map",
    "total_communications": 61,
    "common_map_updates": 14,
    "communication_delay": 0.1,
    "information_gain_percentage_threshold": 5,
    "number_of_agents": 5,
    "start_time": 1652107771.6967163,
    "finish_time": 1652107773.815704,
    "execution_time": 2.118987798690796
}
-------------------------------------------------------------------------------- 

-------------------------------------------------------------------------------- 

{
    "name": "x_square_map",
    "total_communications": 33,
    "common_map_updates": 14,
    "communication_delay": 0.1,
    "information_gain_percentage_threshold": 5,
    "number_of_agents": 5,
    "start_time": 1652107773.8353329,
    "finish_time": 1652107775.194336,
    "execution_time": 1.3590030670166016
}
-------------------------------------------------------------------------------- 

-------------------------------------